In [1]:
import os
import numpy as np
from PIL import Image, ImageTk
from io import BytesIO
import requests
from tkinter import Tk,Label,NW,Canvas,Entry,Button


import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image as image_utils

In [2]:
model = Sequential()


model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu")) #1st input layer
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = "relu")) #1st hidden Layer
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu")) #2nd hidden Layer
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(128, activation="relu")) #output layer
model.add(Dense(2, activation="softmax"))#output laye

In [3]:
model.compile(optimizer="adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

In [4]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip= True)

In [5]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [6]:
training_datasets = train_datagen.flow_from_directory("dataset/training_set", target_size= (128, 128), batch_size = 32, class_mode = "categorical")
testing_datasets = test_datagen.flow_from_directory("dataset/test_set", target_size=(128, 128), batch_size=32, class_mode="categorical")

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [7]:
model.fit(training_datasets, steps_per_epoch=4000/32, epochs=3, validation_data=testing_datasets,validation_steps =  1000/32)

Epoch 1/3
125/125 [==============================] - 78s 611ms/step - loss: 0.6985 - accuracy: 0.5424 - val_loss: 0.6895 - val_accuracy: 0.5645
Epoch 2/3
125/125 [==============================] - 67s 537ms/step - loss: 0.6529 - accuracy: 0.6125 - val_loss: 0.6214 - val_accuracy: 0.6641
Epoch 3/3
125/125 [==============================] - 64s 509ms/step - loss: 0.6170 - accuracy: 0.6685 - val_loss: 0.6224 - val_accuracy: 0.6650


In [8]:
model.save("model_new/model.h5")
model.save_weights("model_new/model_weights.h5")

In [9]:
image_width, image_height = 128, 128
model_path = "model_new/model.h5"
model_wight_path = "model_new/model_weights.h5"

final_model = load_model(model_path) 
final_model.load_weights(model_wight_path)

In [10]:
url = ''
window = Tk()
window.title("Image Classification Using CNN") # plt.title()
window.geometry("800x800")
label = Label(window, text = "Please Enter your Custom URL", font = ("Halvetica", 16))
label.pack()


def Enter():
    global url
    label.configure()
    url = (User_input.get())
    print(url)
    
    
    
    response = requests.get(url)
    test_image = Image.open(BytesIO(response.content))
    put_image = test_image.resize((400, 400))
    test_image = test_image.resize((128, 128))
    
    
    img = ImageTk.PhotoImage(put_image)
    pic = Label(image = img)
    pic.pack()
    
    pic.image = img
    test_image = image_utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    
    results = model.predict_on_batch(test_image)
    
    
    if results[0][0] == 1:
        res = "Cats"
    else:
        res = "Dogs"
    
        
        
    output = Label(window, text = "Predicted Results: "+ res, font = ("Halvetica", 16))
    output.pack()

In [ ]:
User_input = Entry()
User_input.pack()
button = Button(window, text = "Detect Image", font = ("Halvetica", 16), command = Enter)
button.pack()
window.mainloop()

https://d17fnq9dkz9hgj.cloudfront.net/uploads/2012/11/101438745-cat-conjunctivitis-causes.jpg
